# Collaborative filtering recommender

In this notebook, collaborative filtering recommender will be implemented.

The code is adapted fromAndrew Ng's 'Machine Learning Specialiation' course to accomplish the homework assignment. 


# Outline
- [ 1 - Import Packages](#1)
- [ 2 - Goals and Data](#2)
- [ 3 - Main Functions](#3)
- [ 4 - Helper Functions](#4)
- [ 5 - Training](#5)

<a name="1"></a>
## 1 - Import Packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

2023-07-31 20:20:24.676980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<a name="2"></a>
## 2 - Goals and Data 

The goal of a collaborative filtering recommender system is to generate two vectors: For each user, a 'parameter vector' that embodies the movie tastes of a user. For each movie, a feature vector of the same size which embodies some description of the movie. The dot product of the two vectors plus the bias term should produce an estimate of the rating the user might give to that movie.

Once the feature vectors and parameters are learned, they can be used to predict how a user might rate an unrated movie.

Data:
<br>
X (ndarray (num_movies,num_features)): matrix of item features
<br>
W (ndarray (num_users,num_features)) : matrix of user parameters
<br>
b (ndarray (1, num_users)            : vector of user parameters
<br>
Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
<br>
R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user

<a name="3"></a>
## 3 - Main Functions

In [ ]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    lambda_ (float): regularization parameter
    J (float) : Cost
    """
    nm, nu = Y.shape
    nf = X.shape[1]
    J = 0
    wk = 0
    xk = 0

    for i in range (nm): 
        for j in range (nu):
            J += R[i,j] * (np.dot(W[j], X[i]) + b[0,j] - Y[i,j])**2    
            for k in range (nf):
                if i == 0:
                    wk += W[j,k] ** 2
                if j == 0:
                    xk += X[i,k] ** 2
    
    wk = lambda_ /2 * wk
    xk = lambda_ /2 * xk
    J = 1/2 * J + wk + xk  

    return J

In [2]:
# Vectorized Implementation
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

<a name="4"></a>
## 4 - Helper Functions

In [ ]:
def normalizeRatings(Y, R):
    """
    Preprocess data by subtracting mean rating for every movie (every row).
    Only include real ratings R(i,j)=1.
    [Ynorm, Ymean] = normalizeRatings(Y, R) normalized Y so that each movie
    has a rating of 0 on average. Unrated moves then have a mean rating (0)
    Returns the mean rating in Ymean.
    """
    Ymean = (np.sum(Y*R, axis=1)/(np.sum(R, axis=1) + 1e-12)).reshape(-1,1)
    Ynorm = Y - np.multiply(Ymean, R) 
    return(Ynorm, Ymean)

<a name="5"></a>
## 5 - Training

In [ ]:
# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)

#  Useful Values
num_movies, num_users = Y.shape
num_features = 100

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [ ]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")